# Tutorial: Building a Chat Application with Function Calling

- **Level**: Advanced
- **Time to complete**: 20 minutes
- **Components Used**: [InMemoryDocumentStore](https://docs.haystack.deepset.ai/docs/inmemorydocumentstore), [SentenceTransformersDocumentEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder), [SentenceTransformersTextEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformerstextembedder), [InMemoryEmbeddingRetriever](https://docs.haystack.deepset.ai/docs/inmemoryembeddingretriever), [PromptBuilder](https://docs.haystack.deepset.ai/docs/promptbuilder), [OpenAIGenerator](https://docs.haystack.deepset.ai/docs/openaigenerator), [OpenAIChatGenerator](https://docs.haystack.deepset.ai/docs/openaichatgenerator)
- **Prerequisites**: You must have an [OpenAI API Key](https://platform.openai.com/api-keys) and be familiar with [creating pipelines](https://docs.haystack.deepset.ai/docs/creating-pipelines)
- **Goal**: After completing this tutorial, you will have learned how to build chat applications that demonstrate agent-like behavior using OpenAI's function calling feature.

> This tutorial uses Haystack 2.0. To learn more, read the [Haystack 2.0 announcement](https://haystack.deepset.ai/blog/haystack-2-release) or visit the [Haystack 2.0 Documentation](https://docs.haystack.deepset.ai/docs/intro).


## Overview



📚 Useful Sources:
* [OpenAIChatGenerator Docs](https://docs.haystack.deepset.ai/docs/openaichatgenerator)
* [OpenAIChatGenerator API Reference](https://docs.haystack.deepset.ai/reference/generator-api#openaichatgenerator)
* [🧑‍🍳 Cookbook: Function Calling with OpenAIChatGenerator](https://github.com/deepset-ai/haystack-cookbook/blob/main/notebooks/function_calling_with_OpenAIChatGenerator.ipynb)

[OpenAI's function calling](https://platform.openai.com/docs/guides/function-calling) connects large language models to external tools. By providing a `tools` list with functions and their specifications to the OpenAI API calls, you can easily build chat assistants that can answer questions by calling external APIs or extract structured information from text.

In this tutorial, you'll learn how to convert your Haystack pipeline into a function-calling tool and how to implement applications using OpenAI's Chat Completion API through `OpenAIChatGenerator` for agent-like behavior.

## Setting up the Development Environment

Install Haystack 2.0 and [sentence-transformers](https://pypi.org/project/sentence-transformers/) using pip:

In [ ]:
%%bash

pip install haystack-ai
pip install "sentence-transformers>=3.0.0"
pip install arize-phoenix openinference-instrumentation-haystack

### Enable Telemetry

Knowing you're using this tutorial helps us decide where to invest our efforts to build a better product but you can always opt out by commenting the following line. See [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry) for more details.

In [ ]:
from haystack.telemetry import tutorial_running

tutorial_running(40)

Save your OpenAI API key as an environment variable:

In [ ]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

## Add Observability through Phoenix 🐦‍🔥

Phoenix is an LLM observability and evaluation platform that helps developers monitor, evaluate, and improve the performance of their models. We'll use it to instrument our application and collect data on its behavior, and later evaluate the performance of our chatbot.

To learn more about Phoenix, visit the [website](https://phoenix.arize.com/) or [GitHub repo](https://github.com/arize-ai/phoenix).

### Launch the Phoenix App

First we need to launch a local instance of the Phoenix app. Phoenix also provides cloud and self-hosting options you could use instead. If you decide to use one of those hosting options, you can skip the cell below and go straight to [Connecting our application to Phoenix](#connecting-our-application-to-phoenix).

In [ ]:
import phoenix as px
px.launch_app()

### Connect our application to Phoenix

Now we need to connect our notebook to the Phoenix app, and instrument our application. Instrumentation is the process of marking specific parts of our application to be tracked by Phoenix. Phoenix luckily provides an automatic way to instrument Haystack.

In [ ]:
from openinference.instrumentation.haystack import HaystackInstrumentor
from phoenix.otel import register

# Connect to our Phoenix instance:
project_name = "haystack-tutorials-2"
tracer_provider = register(project_name=project_name)

# Instrument Haystack:
HaystackInstrumentor().instrument(tracer_provider=tracer_provider)

Now, any calls to Haystack pipelines will be traced and logged to Phoenix.

# Creating our Agent

## Creating a Function Calling Tool from a Haystack Pipeline

To use the function calling of OpenAI, you need to introduce `tools` to your `OpenAIChatGenerator` using its `generation_kwargs` param.

For this example, you'll use a Haystack RAG pipeline as one of your tools. Therefore, you need to index documents to a document store and then build a RAG pipeline on top of it.

### Index Documents with a Pipeline

Create a pipeline to store the small example dataset in the [InMemoryDocumentStore](https://docs.haystack.deepset.ai/docs/inmemorydocumentstore) with their embeddings. You will use [SentenceTransformersDocumentEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder) to generate embeddings for your Documents and write them to the document store with the [DocumentWriter](https://docs.haystack.deepset.ai/docs/documentwriter).

After adding these components to your pipeline, connect them and run the pipeline.

> If you'd like to learn about preprocessing files before you index them to your document store, follow the [Preprocessing Different File Types](https://haystack.deepset.ai/tutorials/30_file_type_preprocessing_index_pipeline) tutorial.

In [ ]:
from haystack import Pipeline, Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

documents = [
    Document(content="My name is Jean and I live in Paris."),
    Document(content="My name is Mark and I live in Berlin."),
    Document(content="My name is Giorgio and I live in Rome."),
    Document(content="My name is Marta and I live in Madrid."),
    Document(content="My name is Harry and I live in London."),
]

document_store = InMemoryDocumentStore()

indexing_pipeline = Pipeline()
indexing_pipeline.add_component(
    instance=SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"), name="doc_embedder"
)
indexing_pipeline.add_component(instance=DocumentWriter(document_store=document_store), name="doc_writer")

indexing_pipeline.connect("doc_embedder.documents", "doc_writer.documents")

indexing_pipeline.run({"doc_embedder": {"documents": documents}})

### Build a RAG Pipeline

Build a basic retrieval augmented generative pipeline with [SentenceTransformersTextEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformerstextembedder), [InMemoryEmbeddingRetriever](https://docs.haystack.deepset.ai/docs/inmemoryembeddingretriever), [PromptBuilder](https://docs.haystack.deepset.ai/docs/promptbuilder) and [OpenAIGenerator](https://docs.haystack.deepset.ai/docs/openaigenerator).

> For a step-by-step guide to create a RAG pipeline with Haystack, follow the [Creating Your First QA Pipeline with Retrieval-Augmentation](https://haystack.deepset.ai/tutorials/27_first_rag_pipeline) tutorial.

In [ ]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator

template = """
Answer the questions based on the given context.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}
Question: {{ question }}
Answer:
"""
rag_pipe = Pipeline()
rag_pipe.add_component("embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"))
rag_pipe.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store))
rag_pipe.add_component("prompt_builder", PromptBuilder(template=template))
rag_pipe.add_component("llm", OpenAIGenerator(model="gpt-3.5-turbo"))

rag_pipe.connect("embedder.embedding", "retriever.query_embedding")
rag_pipe.connect("retriever", "prompt_builder.documents")
rag_pipe.connect("prompt_builder", "llm")

### Run the Pipeline
Test this pipeline with a query and see if it works as expected before you start using it as a function calling tool.

In [ ]:
query = "Where does Mark live?"
rag_pipe.run({"embedder": {"text": query}, "prompt_builder": {"question": query}})

### Convert the Haystack Pipeline into a Tool

Wrap the `rag_pipe.run` call with a function called `rag_pipeline_func`. This `rag_pipeline_func` function will accept a `query` and return the response coming from the LLM of the RAG pipeline you built before. You will then introduce this function as a tool to your `OpenAIChatGenerator`.

In [ ]:
def rag_pipeline_func(query: str):
    result = rag_pipe.run({"embedder": {"text": query}, "prompt_builder": {"question": query}})

    return {"reply": result["llm"]["replies"][0]}

## Creating Your `tools` List

In addition to the `rag_pipeline_func` tool, create a new tool called `get_current_weather` to be used to get weather information of cities. For demonstration purposes, you can use hardcoded data in this function.

In [ ]:
WEATHER_INFO = {
    "Berlin": {"weather": "mostly sunny", "temperature": 7, "unit": "celsius"},
    "Paris": {"weather": "mostly cloudy", "temperature": 8, "unit": "celsius"},
    "Rome": {"weather": "sunny", "temperature": 14, "unit": "celsius"},
    "Madrid": {"weather": "sunny", "temperature": 10, "unit": "celsius"},
    "London": {"weather": "cloudy", "temperature": 9, "unit": "celsius"},
}


def get_current_weather(location: str):
    if location in WEATHER_INFO:
        return WEATHER_INFO[location]

    # fallback data
    else:
        return {"weather": "sunny", "temperature": 21.8, "unit": "fahrenheit"}

Now, add function specifications for `rag_pipeline_func` and `get_current_weather` to your `tools` list by following [OpenAI's tool schema](https://platform.openai.com/docs/api-reference/chat/create#chat-create-tools). Provide detailed descriptions about `rag_pipeline_func` and `query` so that OpenAI can generate the adaquate arguments for this tool.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "rag_pipeline_func",
            "description": "Get information about where people live",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query to use in the search. Infer this from the user's message. It should be a question or a statement",
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The city and state, e.g. San Francisco, CA"}
                },
                "required": ["location"],
            },
        },
    },
]

## Running OpenAIChatGenerator with Tools

To use the function calling feature, you need to pass the list of tools in the `run()` method of OpenAIChatGenerator as `generation_kwargs`.

Instruct the model to use provided tools with a system message and then provide a query that requires a function call as a user message:

In [ ]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.generators.utils import print_streaming_chunk

messages = [
    ChatMessage.from_system(
        "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    ),
    ChatMessage.from_user("Can you tell me where Mark lives?"),
]

agent = Pipeline()
agent.add_component("chat_generator", OpenAIChatGenerator(model="gpt-3.5-turbo", streaming_callback=print_streaming_chunk))
response = agent._run_component("chat_generator", {"messages":messages, "generation_kwargs":{"tools": tools}})
response

As a response, you'll get a `ChatMessage` with information about the tool name and arguments in JSON format:

```python
{'replies': [
    ChatMessage(
        content='[{"index": 0, "id": "call_3VnT0XQH0ye41g3Ip5CRz4ri", "function": {"arguments": "{\\"query\\":\\"Where does Mark live?\\"}", "name": "rag_pipeline_func"}, "type": "function"}]', role=<ChatRole.ASSISTANT: 'assistant'>, 
        name=None, 
        meta={'model': 'gpt-3.5-turbo-0125', 'index': 0, 'finish_reason': 'tool_calls', 'usage': {}}
        )
    ]
}
```

You can then parse the message content string into JSON and call the corresponding function with the provided arguments.

In [ ]:
import json

## Parse function calling information
function_call = json.loads(response["replies"][0].content)[0]
function_name = function_call["function"]["name"]
function_args = json.loads(function_call["function"]["arguments"])
print("Function Name:", function_name)
print("Function Arguments:", function_args)

## Find the correspoding function and call it with the given arguments
available_functions = {"rag_pipeline_func": rag_pipeline_func, "get_current_weather": get_current_weather}
function_to_call = available_functions[function_name]
function_response = function_to_call(**function_args)
print("Function Response:", function_response)

## Building the Chat Application

As you notice above, OpenAI Chat Completions API does not call the function; instead, the model generates JSON that you can use to call the function in your code. That's why, to build an end-to-end chat application, you need to check if the OpenAI response is a `tool_calls` for every message. If so, you need to call the corresponding function with the provided arguments and send the function response back to OpenAI. Otherwise, append both user and messages to the `messages` list to have a regular conversation with the model. 

To build a nice UI for your application, you can use [Gradio](https://www.gradio.app/) that comes with a chat interface. Install `gradio`, run the code cell below and use the input box to interact with the chat application that has access to two tools you've created above.  

Example queries you can try:
* "***What is the capital of Sweden?***": A basic query without any function calls
* "***Can you tell me where Giorgio lives?***": A basic query with one function call
* "***What's the weather like in Berlin?***", "***Is it sunny there?***": To see the messages are being recorded and sent
* "***What's the weather like where Jean lives?***": To force two function calls
* "***What's the weather like today?***": To force OpenAI to ask more clarification

> Keep in mind that OpenAI models can sometimes hallucinate answers or tools and might not work as expected.

In [ ]:
import json
from typing import List
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack import Pipeline, component

chat_generator = OpenAIChatGenerator(model="gpt-3.5-turbo")
response = None
messages = [
    ChatMessage.from_system(
        "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    )
]

@component
class ToolHandler():
    @component.output_types(final_response=ChatMessage)
    def run(self, messages: List[ChatMessage]):
        # return {"final_response": ChatMessage.from_user("test")}
        response = messages[-1]
        # if OpenAI response is a tool call
        if response and response.meta and response.meta["finish_reason"] == "tool_calls":
            function_calls = json.loads(response.content)
            print(response.content)
            for function_call in function_calls:
                ## Parse function calling information
                function_name = function_call["function"]["name"]
                function_args = json.loads(function_call["function"]["arguments"])

                ## Find the correspoding function and call it with the given arguments
                function_to_call = available_functions[function_name]
                function_response = function_to_call(**function_args)

                ## Append function response to the messages list using `ChatMessage.from_function`
                messages.append(ChatMessage.from_function(content=json.dumps(function_response), name=function_name))
                return self.run(messages)
                
        # Regular Conversation
        else:
            return {"final_response": response}

def chatbot_with_fc(message, history):
    messages.append(ChatMessage.from_user(message))
    agent = Pipeline()
    agent.add_component("chat_generator", OpenAIChatGenerator(model="gpt-3.5-turbo"))
    agent.add_component("tool_handler", ToolHandler())
    agent.connect("chat_generator", "tool_handler")
    
    response = agent.run({"messages": messages, "generation_kwargs": {"tools": tools}})
    
    if response['tool_handler']["final_response"] is None:
        return "no response"
    else:
        return response['tool_handler']["final_response"].content

## Uncomment the line below to launch the chat app with UI
# demo.launch()

In [ ]:
chatbot_with_fc("Can you tell me where Giorgio lives?", [])

# Evaluate our Agent

We can now evaluate any part of our agent. We'll focus on two areas in this example:
* **RAG Pipeline**: Evaluate the performance of the RAG pipeline.
* **Function Calling**: Evaluate the performance of the function calling tool.

We'll use Phoenix for each of these evaluations.

## RAG Pipeline Relevancy Evaluation

All evaluations in Phoenix follow the same process:
1. Export data from your Phoenix project.
2. Run some form of evaluation, either using Phoenix or not.
3. Import the results into Phoenix

In this example, we'll evaluate the relevancy of the documents retrieved by the RAG pipeline.

In [ ]:
import nest_asyncio
import phoenix as px

nest_asyncio.apply()

In [ ]:
from phoenix.session.evaluation import get_retrieved_documents

client = px.Client()

retrieved_documents_df = get_retrieved_documents(client, project_name=project_name)
retrieved_documents_df.head()

In [ ]:
spans_df = client.get_spans_dataframe(project_name=project_name)

def get_input_from_trace_id(trace_id):
    inputs = spans_df.loc[spans_df["context.trace_id"] == trace_id, "attributes.llm.input_messages"].values
    for i in inputs:
        if i is not None:
            return i
    return None

retrieved_documents_df["input"] = retrieved_documents_df["context.trace_id"].map(get_input_from_trace_id)
retrieved_documents_df.head()


In [ ]:
from phoenix.evals import OpenAIModel, RelevanceEvaluator, run_evals

relevance_evaluator = RelevanceEvaluator(OpenAIModel(model="gpt-4o-mini"))

retrieved_documents_relevance_df = run_evals(
    evaluators=[relevance_evaluator],
    dataframe=retrieved_documents_df,
    provide_explanation=True,
    concurrency=20,
)[0]
retrieved_documents_relevance_df.head()

In [ ]:
from phoenix.trace import DocumentEvaluations

px.Client().log_evaluations(
    DocumentEvaluations(dataframe=retrieved_documents_relevance_df, eval_name="Relevance"),
)

## Function Calling Evaluation

Next we can evaluate how good our agent is at calling the right function for a given query.

We'll use the same process as above:
1. Export data from your Phoenix project.
2. Run some form of evaluation, either using Phoenix or not.
3. Import the results into Phoenix

In [ ]:
client = px.Client()

def extract_function_from_output_message(output_message):
    if output_message is not None:
        message_string = str(output_message)
        if "current_weather" in message_string:
            return "get_current_weather"
        elif "rag_pipeline_func" in message_string:
            return "rag_pipeline_func"
    return None

functions_df = client.get_spans_dataframe("'func' in output.value", project_name=project_name)
functions_df = functions_df[["attributes.llm.output_messages", "attributes.llm.input_messages"]]
functions_df["chosen_function"] = functions_df["attributes.llm.output_messages"].apply(extract_function_from_output_message)
functions_df = functions_df.dropna(subset=["chosen_function"])
functions_df = functions_df.rename(columns={"attributes.llm.input_messages": "query"})

functions_df.head()


In [ ]:
llm_as_a_judge_prompt = """
You are an expert evaluator. Your task is to assess the accuracy of function calls made by an AI agent based on a given query. Below is the list of available functions and their descriptions:

{tools_string}

Given the query and the function call made by the agent, evaluate if the chosen function is the most appropriate one. 
Label the chosen function as `correct` if it's the most appropriate one, and `incorrect` if it's not. Additionally, provide an explanation for your evaluation.

Query: {query}
Chosen Function: {chosen_function}

****
Example response:
Label: correct
Explanation: The `rag_pipeline_func` is appropriate for answering questions based on provided documents, which is suitable for the given query.

"""


In [ ]:
from phoenix.evals import (
    OpenAIModel,
    llm_classify
)

eval_model = OpenAIModel(model="gpt-4o-mini")

tools_string = "\n".join([f"{tool['function']['name']}: {tool['function']['description']}" for tool in tools])

llm_as_a_judge_prompt = llm_as_a_judge_prompt.replace("{tools_string}", tools_string)

function_calling_df = llm_classify(
    dataframe=functions_df,
    template=llm_as_a_judge_prompt,
    rails=["correct", "incorrect"],
    model=eval_model,
    provide_explanation=True,
    concurrency=20,
)
function_calling_df["score"] = function_calling_df["label"].apply(lambda x: 1 if x == "correct" else 0)
function_calling_df.head()

In [ ]:
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=function_calling_df, eval_name="Function Calling"),
)


## What's next

🎉 Congratulations! You've learned how to build chat applications that demonstrate agent-like behavior using OpenAI function calling and Haystack Pipelines.

If you liked this tutorial, there's more to learn about Haystack 2.0:
- [Serializing LLM Pipelines](https://haystack.deepset.ai/tutorials/29_serializing_pipelines)
- [Model-Based Evaluation of RAG Pipelines](https://haystack.deepset.ai/tutorials/35_model_based_evaluation_of_rag_pipelines)

To stay up to date on the latest Haystack developments, you can [sign up for our newsletter](https://landing.deepset.ai/haystack-community-updates) or [join Haystack discord community](https://discord.gg/haystack).

Thanks for reading!